[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/stable-cascade-jupyter/blob/main/stable_cascade_gradio_jupyter.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/stable-cascade-hf
%cd /content/stable-cascadeM-hf

!pip install -q git+https://github.com/kashif/diffusers.git@diffusers-yield-callback
!pip install -q accelerate

!python app.py